In [1]:
from feature_eng import bearing, nearest_graph_data, direction
from data import csv_to_df
import osmnx as ox

In [148]:
from numpy import sin,cos,arctan2,pi
def bearing2(c1,c2):
    φ1,λ1 = c1
    φ2,λ2 = c2

    y = sin(λ2-λ1) * cos(φ2)
    x = cos(φ1)*sin(φ2) - sin(φ1)*cos(φ2)*cos(λ2-λ1)
    θ = arctan2(y, x)
    brng = (θ*180/pi + 360) % 360
    return brng

def calc_bearings(df):
    df1 = df
    df2 = df.shift(-1)

    c1 = (df1['lat'], df1['lon'])
    c2 = (df2['lat'], df2['lon'])
    df3 = bearing2(c1, c2)
    df3[df1['speed']<2.0] = None
    df3 = df3.fillna(method='ffill')
    return df3

In [186]:
df = csv_to_df('sample_larger.csv')[::50]
# graph = ox.graph_from_address(
#     'Athens, Municipality of Athens, Regional Unit of Central Athens, Attica, 10667, Greece', 
#     network_type='drive'
# )

In [187]:
df['bearing'] = \
        df.groupby('id', as_index=False, group_keys=False) \
        .apply(calc_bearings)

In [180]:
from numpy import abs, sqrt
def angle_dif(a1,a2):
    # https://gamedev.stackexchange.com/a/4472/86681
    return 180 - abs(abs(a1 - a2) - 180)
# angle_dif(278.447945,345.100550)
def angle_var(angles):
    return ((angles - angles.mean())**2).mean() / len(angles)
def angle_std(angles):
    return np.sqrt(angle_var(angles))
def angle_range(angles):
    return angles.max() - angles.min()

angle_std(df.loc[1]['bearing']), df.loc[1]['bearing']

(0.2179551016582972,
 time
 0.0     321.259404
 2.0     320.379249
 4.0     321.149489
 6.0     319.485488
 8.0     319.869613
 10.0    319.962572
 12.0    320.705310
 14.0    320.605200
 16.0    321.467525
 18.0    321.766784
 20.0    321.992528
 22.0    322.605102
 24.0    321.923424
 26.0    322.449755
 28.0    322.055692
 30.0    320.699926
 32.0    320.699926
 Name: bearing, dtype: float64)

In [158]:

def d_angle(df):
    df1 = df['bearing']
    df2 = df1.shift(-1)
    df3 = (angle_dif(df1,df2) > 20).astype(int)
    df3[-1] = 0
    return df3

df['d_angle'] = df.groupby(['id'], as_index=False, group_keys=False).apply(d_angle)
df[df['d_angle'] == 1]

lat        lon    speed  lon_acc  lat_acc        type  \
id  time                                                                 
17  15.56  37.978563  23.734375  19.2200  -1.6074  -0.9210  Motorcycle   
30  65.44  37.979681  23.733386   0.0063   0.0279   0.0000  Motorcycle   
    69.44  37.979671  23.733394   1.8486   0.2591  -0.1618  Motorcycle   
    71.44  37.979660  23.733369   6.9323   0.0082  -1.1875  Motorcycle   
    83.44  37.979331  23.733613  15.9970  -1.3286  -0.1572  Motorcycle   
...              ...        ...      ...      ...      ...         ...   
496 68.20  37.976734  23.735827  38.4990  -1.4419  -0.7385  Motorcycle   
497 62.16  37.975818  23.735728  20.9802  -1.3425  -0.4747        Taxi   
    64.16  37.975898  23.735718  13.9451   0.3304  -1.4198        Taxi   
498 64.32  37.975806  23.735697  14.8973   0.1558  -0.7794         Car   
    66.32  37.975880  23.735681  16.1562   0.5093  -1.4876         Car   

           traveled_d  avg_speed     bearing  d_angle  
id  time                                               
17  15.56      192.53  34.311641  298.345215        1  
30  65.44      439.34  17.080056  309.779251        1  
    69.44      439.34  17.080056  154.338023        1  
    71.44      439.34  17.080056  206.484817        1  
    83.44      439.34  17.080056  143.590510        1  
...               ...        ...         ...      ...  
496 68.20      523.66  46.894632  357.330451        1  
497 62.16      151.00   9.707244  353.143541        1  
    64.16      151.00   9.707244  318.920197        1  
498 64.32      141.47   8.872855  348.250692        1  
    66.32      141.47   8.872855  318.167758        1  

[493 rows x 10 columns]

In [160]:
df.loc[30]

,lat,lon,speed,lon_acc,lat_acc,type,traveled_d,avg_speed,bearing,d_angle
time,,,,,,,,,,
1.44,37.976960,23.735799,21.0761,1.0197,-0.3035,Motorcycle,439.34,17.080056,324.460257,0
3.44,37.977065,23.735721,25.4939,0.1374,-0.3574,Motorcycle,439.34,17.080056,319.155339,0
5.44,37.977174,23.735623,28.0832,0.2507,-0.3342,Motorcycle,439.34,17.080056,316.123132,0
7.44,37.977286,23.735511,29.5603,0.4822,0.0936,Motorcycle,439.34,17.080056,317.530348,0
9.44,37.977411,23.735392,31.3811,-0.0907,0.1537,Motorcycle,439.34,17.080056,318.668737,0
11.44,37.977540,23.735274,32.7560,0.1427,0.1199,Motorcycle,439.34,17.080056,319.776221,0
13.44,37.977673,23.735157,32.1548,0.0313,-0.1334,Motorcycle,439.34,17.080056,320.166578,0
15.44,37.977809,23.735039,34.5744,0.8353,0.0858,Motorcycle,439.34,17.080056,320.599899,0
17.44,37.977967,23.734904,40.2247,0.3648,-0.0128,Motorcycle,439.34,17.080056,320.539191,0


In [229]:
import numpy as np
df['roll'] = df.groupby('id', as_index=False, group_keys=False).apply(lambda x: x['bearing'].rolling(5, min_periods=1).apply(lambda angles: angle_dif(angles.iloc[-1], angles).max()))
df.loc[498]

,lat,lon,speed,lon_acc,lat_acc,type,traveled_d,avg_speed,bearing,roll
time,,,,,,,,,,
60.32,37.975660,23.735680,15.1436,0.0624,-0.0508,Car,141.47,8.872855,9.592185,0.000000
62.32,37.975734,23.735693,14.4083,0.2698,-0.1240,Car,141.47,8.872855,3.059116,6.533069
64.32,37.975806,23.735697,14.8973,0.1558,-0.7794,Car,141.47,8.872855,348.250692,21.341493
66.32,37.975880,23.735681,16.1562,0.5093,-1.4876,Car,141.47,8.872855,318.167758,51.424426
68.32,37.975952,23.735614,19.1003,0.1207,-1.8440,Car,141.47,8.872855,287.664033,81.928152
70.32,37.975986,23.735503,19.1525,0.5941,-0.0155,Car,141.47,8.872855,276.642291,86.416825
72.32,37.976000,23.735378,19.9480,0.0858,0.2949,Car,141.47,8.872855,277.405096,70.845596
74.32,37.976016,23.735250,20.2328,-0.1928,-0.3768,Car,141.47,8.872855,277.657370,40.510388
76.32,37.976031,23.735134,16.4083,-0.9001,-0.2046,Car,141.47,8.872855,277.689841,9.974192


In [230]:
df_noturn = df[df['roll'] < 20]
df_noturn.loc[400]

,lat,lon,speed,lon_acc,lat_acc,type,traveled_d,avg_speed,bearing,roll
time,,,,,,,,,,
37.64,37.976164,23.736532,51.9420,-2.0150,0.0082,Car,523.65,31.314774,280.260959,0.000000
39.64,37.976207,23.736285,24.7554,-3.5075,-0.0918,Car,523.65,31.314774,280.337827,0.076868
41.64,37.976227,23.736171,15.0898,-0.7296,0.1021,Car,523.65,31.314774,282.603890,2.342931
43.64,37.976247,23.736078,15.4739,-0.5131,0.2868,Car,523.65,31.314774,287.955741,7.694782
45.64,37.976271,23.736001,11.1791,-0.4523,0.2773,Car,523.65,31.314774,298.435951,18.174992
63.64,37.976766,23.735887,19.9843,0.1441,-0.1259,Car,523.65,31.314774,359.459822,8.448877
75.64,37.977492,23.735340,46.8592,1.4482,0.6180,Car,523.65,31.314774,317.679740,4.333574
77.64,37.977700,23.735143,56.7611,1.2739,0.3724,Car,523.65,31.314774,320.234210,6.888044
79.64,37.977953,23.734924,64.5226,0.1991,-0.1492,Car,523.65,31.314774,320.218260,6.872095
